# Exercise 1: Elastic rotating bar

In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton

import ipywidgets as widgets
import numpy as np
import time

In [2]:
def curl(gf: GridFunction)->GridFunction:
    return CoefficientFunction( (0, 0, grad(gf)[1,0]-grad(gf)[0,1]) )

In [3]:
shape = Rectangle(1, 0.1).Face()

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.02))

In [4]:
# density
rho = 1

mass = rho*Integrate(CF(1)*dx, mesh)

# move shape's center of mass to origin
center_of_mass = (1/mass*Integrate(rho*x*dx, mesh), 1/mass*Integrate(rho*y*dx, mesh))

shape = shape.Move((-center_of_mass[0], -center_of_mass[1], 0))

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.02))

# moment_of_inertia = Integrate((x**2 + y**2)*dx, mesh)

# SE(3) translation
r = list(center_of_mass)
r = [0, 0]


In [5]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

## time-stepping

Variables are

* position $\phi = r + R x$, an affine linear function, with constraint $R^T R = I$, i.e. $R$ is orthogonal (and thus, by continuity, a rotation matrix)
* velocity $v = a + b \wedge x$, in body-frame

In [6]:
E, nu = 100, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [7]:
tau = 0.00001
tend = 0.5

# elastic displacement at start
u0 = CF((0, 0))
# elastic displacement velocity at start
v0 = 100*CF((y, -x)) # CF((0, (y+100)))

fes = VectorH1(mesh, order=3)
u,testf = fes.TnT()
aform = InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(testf)))*dx

a = BilinearForm(aform).Assemble()
mstar = BilinearForm(rho*u*testf*dx + tau**2/4*aform).Assemble()
mstarinv = mstar.mat.Inverse(inverse="sparsecholesky")
f = LinearForm(fes).Assemble()

In [8]:
# gfu is displacement?!
gfu = GridFunction(fes)
gfv = GridFunction(fes)
# deformation from displacement
gf_deform = GridFunction(fes)

gfu.Set(u0)
gfv.Set(v0)

gf_graphical = GridFunction(fes)
scene = Draw (gf_graphical, deformation=True, order=3)

# function for debug plot
gfcentrif = GridFunction(fes)
fcentrifx = Draw (gfcentrif[0], mesh, order=3)
fcentrify = Draw (gfcentrif[1], mesh, order=3)

gfrigid = GridFunction(fes)

# pseudo-Newmark (sloppy force calculation)
for j in range(int(tend/tau)):
    
    # centrifugal force
    curl2d = grad(gfv)[1, 0] - grad(gfv)[0, 1]
    f = LinearForm(rho*CF((gfv[1]*curl2d, - gfv[0]*curl2d))*testf*dx).Assemble()
    
    gfu.vec.data += tau/2 * gfv.vec
    gfv.vec.data += tau * mstarinv * (f.vec - a.mat * gfu.vec)
    gfu.vec.data += tau/2 * gfv.vec

    # fit in rigid body translation
    gf_deform.Set(CF((x, y)) + gfu)
    center_of_mass = (1/mass*Integrate(rho*(x + gfu[0])*Norm(Det(Grad(gf_deform)))*dx, mesh), 1/mass*Integrate(rho*(y + gfu[1])*Norm(Det(Grad(gf_deform)))*dx, mesh))
    # print(center_of_mass)
    r[0] = center_of_mass[0]
    r[1] = center_of_mass[1]
    # gfu.Set(gfu - CF(center_of_mass))
    # print(gfv(fem.CoordinateTrafo(CF((x - center_of_mass[0], y - center_of_mass[1])), mesh.Materials(".*"))))
    # gf_deform_old.Set(gfv(fem.CoordinateTrafo(CF((x - center_of_mass[0], y - center_of_mass[1])), mesh.Materials(".*"))))
    # time.sleep(1)
    # gfv.Set(gfv(fem.CoordinateTrafo(CF((x - center_of_mass[0], y - center_of_mass[1])), mesh.Materials(".*"))))
    angle_of_rot = Integrate(InnerProduct(gfu, CF((-y, x)))*dx, mesh)/Integrate(InnerProduct(CF((-y-x, x-y)), CF((-y, x)))*dx, mesh)*pi/2
    R = CF((x * cos(angle_of_rot) - y * sin(angle_of_rot), x * sin(angle_of_rot) + y * cos(angle_of_rot)))
    gfrigid.Set(CF(center_of_mass) + R - CF((x, y)))
    print(angle_of_rot)

    gf_graphical.Set(gfrigid) # CF((r[0], r[1])) + gfrigid

    scene.Redraw()
    
    gfcentrif.Set(gfv)
    fcentrifx.Redraw()
    fcentrify.Redraw()
    
    time.sleep(0.001)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

-0.0015707963267948956
-0.003141589511997394
-0.004712373272448882
-0.006283141325106952
-0.00785388738717313
-0.00942460517622066
-0.010995288410322202
-0.012565930808177512
-0.014136526089241099
-0.01570706797384989
-0.0172775501833505
-0.01884796644022671
-0.02041831046822654
-0.021988575992489174
-0.023558756739671616
-0.025128846438074975
-0.026698838817770357
-0.028268727610724746
-0.029838506550925802
-0.031408169374506544
-0.03297770981986973
-0.034547121627810755
-0.03611639854164113
-0.037685534307310364
-0.039254522673527546
-0.040823357391881976
-0.042392032216963194
-0.04396054090648011
-0.045528877221379514
-0.04709703492596289
-0.048665007788003554
-0.050232789578861736
-0.051800374073599824
-0.053367755051095374
-0.05493492629415445
-0.05650188158962303
-0.058068614728497996
-0.05963511950603741
-0.06120138972186893
-0.06276741918009865
-0.06433320168941795
-0.0658987310632108
-0.06746400111965875
-0.06902900568184742
-0.07059373857787013
-0.07215819364093361
-0.0737223

KeyboardInterrupt: 

In [ ]:
help(Region)

In [ ]:
help(shape.Rotate)

In [ ]:
Integrate(InnerProduct(CF((-y-x, x-y)), CF((-y, x)))*dx, mesh)

In [ ]:
pi/2